In [1]:
import os

os.chdir('../')

## Some analysis

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [3]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [4]:
from Recommenders.SLIM.SLIMElasticNetRecommender_Peppe import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

recommender_slimNet = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train_validation)
recommender_slimNet.fit(topK=142, alpha = 9.521340863590419e-05, l1_ratio = 0.4033590645217344)

recommender_easeR = EASE_R_Recommender(URM_train_validation)
recommender_easeR.fit(l2_norm=33.20)

Progress: 100%|██████████| 38121/38121 [09:02<00:00, 70.24items/s]


EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 15.20 min


In [5]:
W_sparse_EASER = recommender_easeR.W_sparse
W_sparse_EASER = sps.csr_matrix(W_sparse_EASER)
W_sparse_SLIM = recommender_slimNet.W_sparse
print(type(W_sparse_EASER))
print(type(W_sparse_SLIM))

<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>


# Optuna Study

In [6]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

def objective_function(trial):

    alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


    recommender = ItemKNNCustomSimilarityRecommender(URM_train_validation)

    new_similarity = W_sparse_SLIM.multiply(alpha) + W_sparse_EASER.multiply(1-alpha)

    recommender.fit(new_similarity)

    result_dict, _ = evaluator_test.evaluateRecommender(recommender)

    MAP = result_dict.loc[10]["MAP"]
    
    return MAP

In [7]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/Hybrid/hybrid_slim_EASER_MM.csv", index = False)

In [8]:
import optuna

optuna_study = optuna.create_study(study_name="hybrid_slim_ep3", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100
                      )

[I 2024-12-25 12:30:56,630] A new study created in memory with name: hybrid_slim_ep3
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_83744/1623604117.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 2.94 min. Users per second: 202


[I 2024-12-25 12:36:35,093] Trial 0 finished with value: 0.05782048932322598 and parameters: {'alpha': 0.21121052426062592}. Best is trial 0 with value: 0.05782048932322598.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_83744/1623604117.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 2.78 min. Users per second: 213


[I 2024-12-25 12:41:47,770] Trial 1 finished with value: 0.05980981812587179 and parameters: {'alpha': 0.3731075231474956}. Best is trial 1 with value: 0.05980981812587179.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_83744/1623604117.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


...

[I 2024-12-25 21:17:41,055] Trial 97 finished with value: 0.05348922177851594 and parameters: {'alpha': 0.047033528291713034}. Best is trial 91 with value: 0.060664285379833566.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_83744/1623604117.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 2.81 min. Users per second: 211


[I 2024-12-25 21:22:48,968] Trial 98 finished with value: 0.06065873907740571 and parameters: {'alpha': 0.6196575166473326}. Best is trial 91 with value: 0.060664285379833566.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_83744/1623604117.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 2.87 min. Users per second: 207


[I 2024-12-25 21:27:57,833] Trial 99 finished with value: 0.06055491564045933 and parameters: {'alpha': 0.5658948114289528}. Best is trial 91 with value: 0.060664285379833566.


In [9]:
best_index = save_results.results_df["result"].idxmax()
best_hyperparams = save_results.results_df.loc[best_index].to_dict()
print(best_hyperparams)

{'result': 0.060664285379833566, 'alpha': 0.622977719931239}


# Now let's produce the csv

In [5]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender_Peppe import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

recommender_slimNet = MultiThreadSLIM_SLIMElasticNetRecommender(URM_all)
recommender_slimNet.fit(topK=142, alpha = 9.521340863590419e-05, l1_ratio = 0.4033590645217344)

recommender_easeR = EASE_R_Recommender(URM_all)
recommender_easeR.fit(l2_norm=33.20)

Progress: 100%|██████████| 38121/38121 [15:25<00:00, 41.17items/s] 


EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 15.88 min


In [6]:
slim_similarity = recommender_slimNet.W_sparse
easeR_similarity = recommender_easeR.W_sparse
easeR_similarity = sps.csr_matrix(easeR_similarity)

# Saving to disk
sps.save_npz("slim_best_W.npz", slim_similarity)
sps.save_npz("easeR_best_W.npz", easeR_similarity)

In [9]:
alpha = 0.622977719931239
new_similarity = slim_similarity.multiply(alpha) + easeR_similarity.multiply(1-alpha)

final_recommender = ItemKNNCustomSimilarityRecommender(URM_all)
final_recommender.fit(new_similarity)

In [10]:
target_user = pd.read_csv("Dataset/data_target_users_test.csv")
print(target_user.head(10))

user_id = target_user['user_id']
recommendations = []
for user in user_id:
    recommendations.append(final_recommender.recommend(user, cutoff=10))

# Saving in a submission file for the competition
submission = pd.DataFrame(columns=['user_id', 'item_list'])
submission['user_id'] = user_id
# Convert list of recommendations into a space-separated string
submission['item_list'] = [' '.join(map(str, recs)) for recs in recommendations]

# Save the submission file
submission.to_csv(f"submissions/hybrid_slim_EASER_alpha:{alpha}.csv", index=False)

   user_id
0        0
1        1
2        2
3        3
4        4
5        5
6        6
7        7
8        8
9        9
